## 2-4) 연간 앨범 차트에 있는 앨범들의 출시 날짜 크롤링

### yearly_album_chart에 있는 앨범들의 출시 날짜 크롤링

### 1. 정확한 검색을 위한 데이터 전처리

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import re
import numpy as np

In [13]:
yearly_album_chart_df = pd.read_csv('data/yearly_album_chart_df.csv')
len(yearly_album_chart_df)

500

- 정확한 검색을 위해 따옴표 하나로 처리

In [15]:
# yearly_album_chart_df['title'] = [title.replace("''", "'") for title in yearly_album_chart_df['title']]

* 정확한 검색을 위해 대괄호 삭제

In [20]:
# new_title_list = []
# for title in yearly_album_chart_df['title']:
#    new_title_list.append(re.sub(pattern=r'\([^)]*\)', repl='', string = title).strip())

# yearly_album_chart_df['title'] = new_title_list

In [23]:
# new_singer_list = []
# for singer in yearly_album_chart_df['singer']:
#    new_singer_list.append(re.sub(pattern=r'\([^)]*\)', repl='', string = singer).strip())

# yearly_album_chart_df['singer'] = new_singer_list

* 확인

In [31]:
yearly_album_chart_df.head(5)

,year,rank,title,singer,sales
0,2018,1,LOVE YOURSELF 結 'Answer',방탄소년단,2197808
1,2018,2,LOVE YOURSELF 轉 'Tear',방탄소년단,1849537
2,2018,3,DON'T MESS UP MY TEMPO - The 5th Album,EXO,1452030
3,2018,4,0+1=1,Wanna One,782562
4,2018,5,1÷χ=1,Wanna One,641131


In [ ]:
# yearly_album_chart_df.loc[yearly_album_chart_df['title'] == '1÷χ=1', 'title'] = '1÷x=1'

### 2. 앨범 출시 날짜 크롤링 하기

- 검색 url 생성 함수 만들기 <br>
create_searchAlbum_url(idx)<br>
검색명: 앨범명 + 가수이름

In [44]:
#first_url = 'https://www.genie.co.kr/search/searchAlbum?query=LOVE%2520YOURSELF%2520%25E7%25B5%2590%2520%27Answer%27%2520%25EB%25B0%25A9%25ED%2583%2584%25EC%2586%258C%25EB%2585%2584%25EB%258B%25A8'

In [47]:
# driver_path = 'C:/Users/Playdata/Desktop/dev/setup/chromdriver.exe'
driver_path = 'C:/Users/k9942/OneDrive/바탕 화면/playdata/dev/setup/chromdriver.exe'
driver = webdriver.Chrome(driver_path)

C:\Users\k9942\AppData\Local\Temp\ipykernel_9236\3454815965.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


In [108]:
def create_searchAlbum_url(idx):
    title_singer = list(yearly_album_chart_df.loc[idx][2:4]) # title + singer
    search_name = ' '.join(title_singer)

    input_search_name = driver.find_elements(By.CSS_SELECTOR, 'input.ipt-search')[0]
    input_search_name.clear()
    input_search_name.send_keys(search_name)

    button = driver.find_element(By.CSS_SELECTOR, 'input.btn-submit')
    button.click()

    now_url = driver.current_url
    search_url = now_url.replace('searchMain', 'searchAlbum')

    return search_url

* release_date 크롤링

50개씩 크롤링 구간 설정) <br>
-- 1. range(100) <br>
-- 2. range(100, 200) <br>
-- 3. range(200, 300) <br>
-- 4. range(300, 400) <br>
-- 5. range(400, 500) <br>

In [132]:
# 5 구간 크롤링
release_date_list_500 = []

for idx in range(400, 500):
    search_url = create_searchAlbum_url(idx)

    driver.get(search_url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    release_date = soup.select('dd.desc')

    try:
        release_date = release_date[0].get_text().split('|')[0].strip()

    except:
        release_date = np.NaN

    release_date_list_500.append(release_date)
    time.sleep(3)

len(release_date_list_500)

100

In [133]:
# 데이터 프레임 생성
release_data_500_df = pd.DataFrame()
release_data_500_df['release_date'] = release_date_list_500

In [134]:
# null 값 확인
release_data_500_df.isnull().sum()

release_date    0
dtype: int64

In [135]:
# shape 확인
release_data_500_df.shape

(100, 1)

In [136]:
# 5구간 df를 csv 파일로 저장
release_data_500_df.to_csv('data/release_data_500_df.csv', encoding='utf-8', index=False)

- total_release_date_df 생성

In [143]:
df_100 = pd.read_csv('data/release_data_100_df.csv')
df_200 = pd.read_csv('data/release_data_200_df.csv')
df_300 = pd.read_csv('data/release_data_300_df.csv')
df_400 = pd.read_csv('data/release_data_400_df.csv')
df_500 = pd.read_csv('data/release_data_500_df.csv')

total_release_date_df = pd.concat([df_100, df_200, df_300, df_400, df_500])
total_release_date_df.shape

(500, 1)

In [144]:
total_release_date_df.head()

,release_date
0,2018.08.24
1,2018.05.18
2,2018.11.02
3,2018.03.19
4,2018.06.04


In [145]:
total_release_date_df.tail()

,release_date
95,2021.05.31
96,2022.05.24
97,2022.03.15
98,2022.10.19
99,2022.08.16


In [147]:
total_release_date_df.isnull().sum()

release_date    8
dtype: int64

In [150]:
# 500개의 release_date 저장
total_release_date_df.to_csv('data/total_release_date_df.csv', encoding='utf-8', index=False)

### 3. yearly_album_chart_with_date.csv 생성

In [151]:
yearly_album_chart = pd.read_csv('data/yearly_album_chart_df.csv')
yearly_album_chart.shape

(500, 5)

In [157]:
total_release_date_list = list(total_release_date_df['release_date'])
len(total_release_date_list)

500

In [159]:
yearly_album_chart['release_date'] = total_release_date_list

In [160]:
yearly_album_chart.shape

(500, 6)

In [161]:
yearly_album_chart.head()

,year,rank,title,singer,sales,release_date
0,2018,1,LOVE YOURSELF 結 ''Answer'',방탄소년단,2197808,2018.08.24
1,2018,2,LOVE YOURSELF 轉 ''Tear'',방탄소년단,1849537,2018.05.18
2,2018,3,DON''T MESS UP MY TEMPO - The 5th Album,EXO,1452030,2018.11.02
3,2018,4,0+1=1 (I PROMISE YOU),Wanna One (워너원),782562,2018.03.19
4,2018,5,1÷χ=1 (UNDIVIDED),Wanna One (워너원),641131,2018.06.04


In [162]:
yearly_album_chart.tail()

,year,rank,title,singer,sales,release_date
495,2022,96,혼돈의 장: FREEZE,투모로우바이투게더,152900,2021.05.31
496,2022,97,The Story (META),강다니엘,149807,2022.05.24
497,2022,98,REFUGE,문빈&산하,147056,2022.03.15
498,2022,99,Love Pt.2 : Passion,위아이 (WEi),146044,2022.10.19
499,2022,100,7TH MINI ALBUM [BE AWARE] (META),더보이즈(THE BOYZ),144734,2022.08.16


In [163]:
yearly_album_chart.to_csv('data/yearly_album_chart_with_date.csv', encoding='utf-8', index=False)

----